In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Загрузка данных

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [3]:
y = data['price']
X = data.drop(columns='price')

# Предобработка

In [4]:
#lr = LinearRegression()
#lr.fit(X,y)

_______________________
Сторонний пример

In [5]:
from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
 
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
lb.fit(education)
 
print('категории:', lb.classes_) 
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

___________________________________

In [6]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений призкака {}: '.format(column), data[column].nunique())

Число уникальных значений призкака cylinders:  6
Число уникальных значений призкака title_status:  5
Число уникальных значений призкака transmission:  3
Число уникальных значений призкака drive:  3
Число уникальных значений призкака size:  4


In [7]:
from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray() 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


In [9]:
data_onehot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

✍ Как вы уже знаете, большинство моделей машинного обучения не умеют работать с текстовыми признаками.

→ Скачайте ноутбук с кодом юнита

Обратимся к нашим данным. Нас просят предсказать цену автомобиля. Для обучения модели выделим целевой столбец:

y = data['price']
x = data.drop(columns='price')
Попробуем применить линейную регрессию на «сырых» данных:

lr = LinearRegression()
lr.fit(x,y)
Мы получим ошибку с комментарием о том, что не удалось превратить строковое значение в число с плавающей точкой (float).

img

Чтобы этой ошибки не возникало, необходимо закодировать данные.

В разделе EDA-3 вы познакомились с такими методами кодирования, как:

порядковое кодирование;
двоичное кодирование;
однократное кодирование.
img

В таблице ниже представлено сравнение данных способов кодировки:

Значения признака «Образование»	Порядковое кодирование	Двоичное кодирование	Однократное кодирование
Нет	1	000	000001
Начальное	2	001	000010
Среднее	3	010	000100
BSc	4	011	001000
MSc	5	100	010000
PhD	6	101	100000
Задание 3.1
2/2 points (graded)
Выберите кодировку, которая соответствует иллюстрации:

img


Порядковая
 верно 
img


Двоичная
 верно 
Show answer
Отправить
Some problems have options such as save, reset, hints, or show answer. These options follow the Submit button.
Задание 3.2
1/1 point (graded)
Какая цифра будет стоять в последнем столбце для строки с образованием PhD при однократном кодировании?
0
  верно 
 
Show answer
Отправить
Some problems have options such as save, reset, hints, or show answer. These options follow the Submit button.
Для реализации данных методов мы использовали библиотеку category_encoders. Однако вы уже знаете и такой мощный инструмент, как scikit-learn. Данная библиотека содержит набор реализованных алгоритмов машинного обучения, метрик для оценки их качества, а также  класс preprocessing для предобработки данных, в частности — для кодирования категориальных признаков.

Представленная ниже таблица показывает соответствие типа кодирования классу в sklearn.processing.

Тип кодировки	Класс библиотеки sklearn
Порядковое кодирование	LabelEncoder
Двоичное кодирование	LabelBinarizer
Однократное кодирование	OneHotEncoder
img

Преимущество использования одной библиотеки состоит в типичности методов. Например, вы знаете, что для обучения модели в sklearn используется метод fit(). При кодировании признаков здесь также применяют fit() для подгонки кодировщика под выборку и transform() — для преобразования данных в числа.

Алгоритм кодирования в sklearn следующий:

img

Из предыдущих модулей мы знаем, что при решении задач машинного обучения данные разбираются на обучающую (train) и валидационную (validation) выборки (последняя также может быть тестовой (test) выборкой). По аналогии подгонка кодировщика происходит на обучающей выборке, а трансформация — на обучающей и на тестовой.

Почему так? Потому что наша обученная модель не должна видеть данные, которые подаются в неё на тесте. Только так мы можем судить о том, что модель обучена качественно. То же самое и с кодировкой.

Давайте посмотрим на кодирование признака Образование способом «один-против-всех» (one vs all):

from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
 
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
 
lb.fit(education)
 
print('категории:', lb.classes_)
 
lb.transform(['нет', 'MSc'])
img

У класса LabelBinarizer, как и у двух остальных, есть атрибут classes_, который выводит список уникальных значений признака.

Вернёмся к нашей выборке. В ней присутствуют следующие категориальные признаки: condition, cylinders, title_status, transmission, drive, size.

При этом признаки condition и cylinders — числовые, а title_status, transmission, drive, size — текстовые.

img

Задание 3.3
1/1 point (graded)
Какой тип имеет признак condition?
Порядковый
Номинальный
верно
Ответ
Верно:Это порядковый признак, так как состояние машины 0 явно хуже состояния 5.
Show answer
Отправить
Some problems have options such as save, reset, hints, or show answer. These options follow the Submit button.
Задание 3.4
1/1 point (graded)
Какой тип имеет признак cylinders?
Порядковый
Номинальный
верно
Ответ
Верно:Это номинальный признак: в нём нет градации значений.
Show answer
Отправить
Some problems have options such as save, reset, hints, or show answer. These options follow the Submit button.Верно (1/1 балл)Review
Задание 3.5
1/1 point (graded)
Нужно ли преобразовывать признаки cylinders и condition?
Да, нужно преобразовать оба признака.
Нет, не нужно преобразовывать ни один из признаков.
Нужно преобразовать только cylinders.
Нужно преобразовать только condition.
верно
Ответ
Верно:condition — это порядковый признак, который уже представлен в числовом формате, его кодировать не нужно. cylinders — номинальный признак, его нужно закодировать.
Show answer
Отправить
Some problems have options such as save, reset, hints, or show answer. These options follow the Submit button.Верно (1/1 балл)Review
Важно производить кодирование номинальных признаков, даже если они уже представлены в числовом формате, так как, в отличие от порядковых признаков, категории номинальных являются независимыми. В случае порядкового кодирования таких признаков мы вносим искусственные закономерности в данные (например, чем больше числовой код цилиндров, тем лучше, хотя это необязательно так).

Задание 3.6
1/1 point (graded)
Какой способ кодирования следует выбрать для номинального признака в случае небольшого числа категорий?
Порядковый
Однократный
верно
Ответ
Верно:Однократное кодирование — выбор для номинальных признаков, если их меньше 15.
Show answer
Отправить
Some problems have options such as save, reset, hints, or show answer. These options follow the Submit button.Верно (1/1 балл)Review
Посмотрим на число уникальных значений номинальных признаков title_status, transmission, drive, size и cylinders:

columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений признака {}: '.format(column), data[column].nunique())
img

Итак, нам подходит однократное кодирование. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — fit_transform(). В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода get_feature_names_out() получим список новых названий колонок:

from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()
 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)
img

Отлично, категориальные признаки закодированы. Теперь нам необходимо соединить преобразованные данные с исходными.

Напомним, что у библиотеки pandas есть дефолтный метод get_dummies() для получения однократного кодирования признаков. Однако OneHotEncoder способен принимать на вход как таблицы, так и numpy-массивы.